几乎素有有用的程序都会设计到么哦谢文本处理，不管是解析数据还是产生输出。这一章将重点关注文本的操作处理，比如提取字符串，搜索，替换以及解析等。大部分的问题都能简单的调用字符串的内建方法完成。但是，一些更为复杂的操作可能需要正则表达式或者强大的解析器，所有这些主题我们都会详细讲解。并且在操作Unicode时候碰到的一些棘手问题在这里也会被提及到。

## 1. 使用多个界定符分割字符串

string对象的split（）方法只适用于非常简单的字符串分割情形，它并不允许有多个分隔符或者分隔符周围不确定的空格。当你需要更加灵活的切割字符串的时候，最好使用re.split()方法：

In [1]:
line = 'asdf fjdk; afed, fjek,asdf, foo'

In [2]:
import re

In [3]:
re.split(r'[;,\s]\s*',line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

>> 函数re.split()是非常实用的，因为它允许你为分隔符指定多个正则模式。比如上面的例子，分隔符可以是逗号，分号或者空格，并且后面紧跟着任意个空格。只要这个模式被找到，那么匹配的分隔符两边的实体都会被当成结果中的元素返回。返回的结果为一个字段列表，这个跟str.split()返回值类型是一样的。  
>> 当你使用re.split()函数时候，需要特别注意的是正则表达式中是否包含一个括号捕获分组。如果使用了捕获分组，那么被匹配的文本将出现在结果列表中。比如下面这样：

In [4]:
fields = re.split(r'(;|,|\s)\s*', line)
fields

['asdf', ' ', 'fjdk', ';', 'afed', ',', 'fjek', ',', 'asdf', ',', 'foo']

获取分割字符在某些情况下也是有用的。 比如，你可能想保留分割字符串，用来在后面重新构造一个新的输出字符串：

In [5]:
values = fields[::2]

In [6]:
delimiters = fields[1::2]+['']

In [7]:
values

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

In [8]:
delimiters

[' ', ';', ',', ',', ',', '']

In [9]:
#reform the line using the same delimiters
''.join(v+d for v,d in zip(values,delimiters))

'asdf fjdk;afed,fjek,asdf,foo'

如果你不想保留分割字符串到结果列表中去，但仍然需要使用到括号来分组正则表达式的话， 确保你的分组是非捕获分组，形如 (?:...) 。比如：

In [10]:
re.split(r'(?:,|;|\s)\s*',line)

['asdf', 'fjdk', 'afed', 'fjek', 'asdf', 'foo']

## 2. 字符串开头或结尾匹配

In [11]:
filename = 'spam.txt'

In [12]:
filename.endswith('.txt')

True

In [13]:
filename.startswith('file:')

False

In [14]:
url = 'http://www.python.org'

In [15]:
url.startswith('http:')

True

如果你想检查多种匹配可能，只需要将所有的匹配项放入到一个元组中去，然后传给startswith()或者endswith()方法。

In [16]:
import os

In [17]:
filename = os.listdir('.')

In [18]:
filename

['第一章： 数据结构和算法.ipynb',
 'somefile.txt',
 '第二章：字符串和文本.ipynb',
 '.ipynb_checkpoints']

In [19]:
[name for name in filename if name.endswith('.txt')]

['somefile.txt']

In [20]:
any(name.endswith('.py') for name in filename)

False

In [21]:
# 下面是另一个例子
from urllib.request import urlopen

def read_data(name):
    if name.startswith(('http:','https:','ftp:')):
        return urlopen(name).read()
    else:
        with open(name) as f:
            return f.read()

奇怪的是，这个方法必须要输入一个元组作为参数。如果你恰巧又一个list或者set类型的选择项，要确保传递参数前调用tuple()将其转换为元组类型。比如：

In [22]:
choices = ['http:','ftp:']

In [23]:
url = 'http://www.python.org'

In [24]:
url.startswith(choices)

TypeError: startswith first arg must be str or a tuple of str, not list

In [ ]:
url.startswith(tuple(choices))

>> startswith()和endswith()方法提供了一个非常方便的方式去做字符串开头和结尾的检查。类似的操作也可以使用切片来实现，但是代码看起来没有那么优雅。

In [ ]:
filename = 'spam.txt'

In [ ]:
filename[-4:] == '.txt'

In [ ]:
url = 'http://www.python.org'

In [ ]:
url[:5] == 'http:' or url[:6] == 'https:' or url[4:] == 'ftp:'

In [ ]:
# 还可以使用正则表达式来实现
import re
url = 'http://www.python.org'
re.match('http:|https:|ftp:',url)

## 3. 用shell通配符匹配字符串

In [ ]:
# fnmatch模块提供了两个函数--fnmatch ()和fnmatchcase(),可以用来匹配字符串
from fnmatch import fnmatch,fnmatchcase
fnmatch('foo.txt','*.txt')

In [ ]:
fnmatch('foo.txt','?oo.txt')

In [ ]:
fnmatch('Dat45.csv','Dat[0-9]*')

In [ ]:
names = ['Dat1.csv','Dat2.csv','config.ini','foo.py']

In [ ]:
[name for name in names if fnmatch(name,'Dat*.csv')]

In [ ]:
# fnmatch()函数使用底层操作系统的大小写敏感规则（不同的系统是不一样的）来匹配模式。比如：
# On OS X (Mac)
fnmatch('foo.txt','*.TXT')

In [ ]:
# 如果你对这个区别很在意，可以使用fnmatchcase()来代替。它完全使用你的模式大小写匹配
fnmatchcase('foo.txt','*.TXT')

这两个函数通常会被忽略的一个特性是在处理非文件名的字符串时候它们也是很有用的。 比如，假设你有一个街道地址的列表数据：

In [ ]:
addresses = [
    '5412 N CLARK ST',
    '1060 W ADDISON ST',
    '1039 W GRANVILLE AVE',
    '2122 N CLARK ST',
    '4802 N BROADWAY',
]

In [ ]:
# 你可以想这样写列表推导
from fnmatch import fnmatchcase
[addr for addr in addresses if fnmatchcase(addr,'* ST')]

>> fnmatch()函数匹配能力介于简单的字符串和强大的正则表达式之间。如果你在数据操作中只需要简单的通配符就能完成的时候，这通常是一个比较合理的方案

## 4. 字符串匹配和搜索

In [ ]:
# 如果你想匹配的是字面字符串，那么你通常只需要调用基本的此付出方法就行。比如，str.find(),
# str.endswith(),str.startswith()或者类似的方法。
text = 'yeah, but no, but yeah, but no,but yeah'

In [ ]:
# exact match
text == 'yeah'

In [ ]:
# match at start or end
text.startswith('yeah')

In [ ]:
text.endswith('yerh')

In [ ]:
# search for the locateion of the first occurence
text.find('no')

对于复杂的匹配需要使用正则表达式和re模块。为了解释正则表达式的基本原理，假设你想匹配数字格式的日期字符串比如11/27/2012,你可以这样做：

In [25]:
text1 = '11/27/2012'

In [26]:
text2 = 'Nov 27, 2012'

In [27]:
import re

In [28]:
if re.match(r'\d+/\d+/\d+',text1):
    print('yes')
else:
    print('no')

yes


In [29]:
if re.match(r'\d+/\d+/\d+',text2):
    print('yes')
else:
    print('no')

no


In [30]:
#如果你想使用同一个模式去做多次匹配，你应该先将模式字符串预编译为模式对象。
datepat = re.compile(r'\d+/\d+/\d+')
if datepat.match(text1):
    print('yes')
else:
    pritn('no')

yes


In [31]:
if datepat.match(text2):
    print('yes')
else:
    print('no')

no


match()总是从字符串开始去匹配，如果你想查找字符串热议部分的模式出香位置，使用findall()方法去代替。

In [32]:
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'

In [33]:
datepat.findall(text)

['11/27/2012', '3/13/2013']

In [34]:
# 在定义正则式的时候，通常会利用括号去捕获分组。比如
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')

In [35]:
# 捕获分组可以使得后面的处理更加简单，因为可以分别将每个组的内容提取出来。
m = datepat.match('11/27/2012')
m

<re.Match object; span=(0, 10), match='11/27/2012'>

In [36]:
# extract the content of each group
m.group(0)

'11/27/2012'

In [37]:
m.group(1)

'11'

In [38]:
m.group(2)

'27'

In [39]:
m.group(3)

'2012'

In [40]:
m.groups()

('11', '27', '2012')

In [41]:
month,day,year = m.groups()

In [42]:
# find all matches (notice spliting into tuples)
text

'Today is 11/27/2012. PyCon starts 3/13/2013.'

In [43]:
datepat.findall(text)

[('11', '27', '2012'), ('3', '13', '2013')]

In [44]:
for month,day,year in datepat.findall(text):
    print('{}-{}-{}'.format(year,month,day))

2012-11-27
2013-3-13


findall()方法会搜索文本并以列表的形式返回的匹配。如果你想迭代方式返回匹配，可以使用finditer()方法来代替，比如：

In [45]:
for m in datepat.finditer(text):
    print(m.groups())

('11', '27', '2012')
('3', '13', '2013')


## 5. 字符串搜索和替换

In [46]:
# 对于简单的字面模式，直接使用str.replace()方法即可
text = 'yeah, but no, but yeah, but no, but yeah'
text.replace('yeah','yep')

'yep, but no, but yep, but no, but yep'

In [47]:
# 对于复杂的模式，请使用re模块的sub()函数。
text = 'Today is 11/27/2012. PyCon starts 3/13/2013.'

In [48]:
import re
re.sub(r'(\d+)/(\d+)/(\d+)',r'\3-\1-\2',text)

'Today is 2012-11-27. PyCon starts 2013-3-13.'

sub()函数中的第一个参数是被匹配的模式，第二个参数是替换模式。反斜杠数字\3指向前面模式的捕获组号

In [49]:
#如果你打算用相同的模式做多次替换，考虑先编译它来提升性能。比如：
import re
datepat = re.compile(r'(\d+)/(\d+)/(\d+)')

In [50]:
datepat.sub(r'\3-\1-\2',text)

'Today is 2012-11-27. PyCon starts 2013-3-13.'

In [51]:
# 对于更加复杂的替换，可以传递一个替换回调函数来代替，比如：
from calendar import month_abbr
def change_date(m):
    mon_name = month_abbr[int(m.group(1))]
    return '{} {} {}'.format(m.group(2),mon_name,m.group(3))

In [52]:
datepat.sub(change_date,text)

'Today is 27 Nov 2012. PyCon starts 13 Mar 2013.'

一个替换回调函数的参数是一个match对象，也就是match（）或者find（）返回的对象。使用group()方法来提取特定的匹配部分。回调函数最后返回替换字符串。

In [53]:
# 如果除了替换后的结果外，你还想知道有多少替换发生了，可以使用re.subn()来代替。
newtext,n = datepat.subn(r'\3-\1-\2',text)

In [54]:
newtext

'Today is 2012-11-27. PyCon starts 2013-3-13.'

In [55]:
n

2

## 6. 字符串忽略大小写的搜索替换

In [56]:
# 为了在文本操作时忽略大小写，你需要在使用re模块的时候被这些提供re.INGORECASE标志参数
text = 'UPPER PYTHON, lower python, Mixed Python'
re.findall('python',text,flags=re.IGNORECASE)

['PYTHON', 'python', 'Python']

In [57]:
re.sub('python','snake',text,flags=re.IGNORECASE)

'UPPER snake, lower snake, Mixed snake'

最后那个例子揭示了一个小缺陷，替换字符串并不会自动跟匹配的字符串的大小写保持一致。为了修复这个，你可能需要一个辅助函数，就像下面的这样：

In [58]:
def matchcase(word):
    def replace(m):
        text = m.group()
        if text.isupper():
            return word.upper()
        elif text.islower():
            return word.lower()
        elif text[0].isupper():
            return word.capitalize()
        else:
            return word
    return replace

## 7. 最短匹配模式

解决方案： 这个问题一般出现在需要匹配一对分隔符之间的文本的时候（比如引号的字符串）。

In [59]:
str_pat = re.compile(r'"(.*)"')

In [60]:
text1 = 'Computer says "no."'

In [61]:
str_pat.findall(text1)

['no.']

In [62]:
text2 = 'Computer says "no." Phone says "yes."'

In [63]:
str_pat.findall(text2)

['no." Phone says "yes.']

在这个例子中，模式r'\"l(.*)\"'的意图是i 匹配被双引号剥好的文本。但是在正则表达式中的*号操作符是贪婪的，因此匹配操作会查找最长的可能匹配。于是在第二个例子中搜索text2的时候返回结果并不是我们想要的

In [64]:
# 为了修正这个问题，可以在模式中的*操作符后面加上？修饰符，就像这样的：
str_pat = re.compile(r'"(.*?)"')

In [65]:
str_pat.findall(text2)

['no.', 'yes.']

这样就使得匹配变成了非贪婪模式，从而得到最短的匹配，也就是我们想要的结果了。

## 8. 多行匹配模式

In [66]:
commet = re.compile(r'/\*(.*?)\*/')

In [67]:
text1 = '/* this is a comment */'

In [68]:
text2 = '''/* this is a 
    multiline comment */'''

In [69]:
commet.findall(text1)

[' this is a comment ']

In [70]:
commet.findall(text2)

[]

In [71]:
# 为了修正这个问题，可以修改模式字符串，增加对换行的支持。比如：
comment = re.compile(r'/\*(?:.|\n)*?\*/')
comment.findall(text2)

['/* this is a \n    multiline comment */']

在这个模式中， (?:.|\n) 指定了一个非捕获组 (也就是它定义了一个仅仅用来做匹配，而不能通过单独捕获或者编号的组)。

re.compile() 函数接受一个标志参数叫 re.DOTALL ，在这里非常有用。 它可以让正则表达式中的点(.)匹配包括换行符在内的任意字符。比如：

In [72]:
comment = re.compile(r'/\*(.*?)\*/',re.DOTALL)

In [73]:
comment.findall(text2)

[' this is a \n    multiline comment ']

>> 对于简单的情况使用 re.DOTALL 标记参数工作的很好， 但是如果模式非常复杂或者是为了构造字符串令牌而将多个模式合并起来(2.18节有详细描述)， 这时候使用这个标记参数就可能出现一些问题。 如果让你选择的话，最好还是定义自己的正则表达式模式，这样它可以在不需要额外的标记参数下也能工作的很好。

## 9. 将Unicode文本标准化

Unicode中，某些字符能够用多个合法的编发表示。为了说明考虑下面的例子：

In [74]:
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalapen\u0303o'

In [75]:
s1

'Spicy Jalapeño'

In [76]:
s2

'Spicy Jalapeño'

In [77]:
s1 == s2

False

In [78]:
len(s1)

14

In [79]:
len(s2)

15

这里的文本”Spicy Jalapeño”使用了两种形式来表示。 第一种使用整体字符”ñ”(U+00F1)，第二种使用拉丁字母”n”后面跟一个”~”的组合字符(U+0303)。



In [80]:
import unicodedata

In [81]:
t1 = unicodedata.normalize('NFC',s1)

In [82]:
t2 = unicodedata.normalize('NFC',s2)

In [83]:
t1 == t2

True

In [84]:
print(ascii(t1))

'Spicy Jalape\xf1o'


>> 标准化杜宇任何需要以一致的方式处理Unicode文本的程序都是非常重要的。当处理来自用户输入的字符串而你很难去控制编码的时候有趣如此。

## 10. 在正则式中使用Unicode

默认情况下re模式已经对一些Unicode字符类有了基本的支持。比如，\\d已经匹配任意的unicode数字字符了：

In [85]:
import re

In [86]:
num = re.compile('\d+')

In [87]:
# ASCII digits
num.match('123')

<re.Match object; span=(0, 3), match='123'>

In [88]:
# Arabic digits
num.match('\u0661\u0662\u0663')

<re.Match object; span=(0, 3), match='١٢٣'>

如果你想在模式中包含指定的Unicode字符，你可以使用Unicode字符对应的转义序列(比如 \uFFF 或者 \UFFFFFFF )。 比如，下面是一个匹配几个不同阿拉伯编码页面中所有字符的正则表达式：

In [89]:
arabic = re.compile('[\u0600-\u06ff\u0750-\u077f\u08a0-\u08ff]+')
arabic

re.compile(r'[\u0600-ۿݐ-ݿࢠ-ࣿ]+', re.UNICODE)

>> 混合使用Unicode和正则表达式通常会让你抓狂。 如果你真的打算这样做的话，最好考虑下安装第三方正则式库， 它们会为Unicode的大小写转换和其他大量有趣特性提供全面的支持，包括模糊匹配。

## 11. 删除字符串中不需要的字符

strip()方法能用于删除开始或结尾的字符。lstrip()和rstrip()分别从左和从右执行删除操作。默认情况下，这些方法会去除空白字符，但是你也可以指定其他字符

In [90]:
# Whitespace stripping
s = ' hello world\n'
s.strip()

'hello world'

In [91]:
s.lstrip()

'hello world\n'

In [92]:
s.rstrip()

' hello world'

In [93]:
# Character stripping
t = '-----hello======='

In [94]:
t.lstrip('-')

'hello======='

In [95]:
t.strip('-=')

'hello'

>> 这些strip方法在读取和清理数据以备后续处理的时候是经常会被用到的。比如，你可以用他们来去除空格，引号和完成其他的任务。

In [96]:
# 但是需要注意的是，去除操作不会对自堵车的中间文本产生任何影响。
s = ' hello    world\n'

In [97]:
s = s.strip()
s

'hello    world'

>> 如果你想处理中间的空格，那么你需要求助其他技术。比如使用replace()方法或者是用正则表达式替换。

In [98]:
# first method
s.replace(' ','')

'helloworld'

In [99]:
# second method
import re

re.sub('\s+',' ',s)

'hello world'

通常情况下你想将字符串 strip 操作和其他迭代操作相结合，比如从文件中读取多行数据。 如果是这样的话，那么生成器表达式就可以大显身手了。比如：

In [100]:
# with open(filename) as f:
#    lines = (line.strip() for line in f)
#    for line in lines:
#       print(line)

在这里，表达式 lines = (line.strip() for line in f) 执行数据转换操作。 这种方式非常高效，因为它不需要预先读取所有数据放到一个临时的列表中去。 它仅仅只是创建一个生成器，并且每次返回行之前会先执行 strip 操作。

## 12. 审查清理文本字符串

文本清理问题会涉及到包括文本解析与数据处理等一系列问题。 在非常简单的情形下，你可能会选择使用字符串函数(比如 str.upper() 和 str.lower() )将文本转为标准格式。 使用 str.replace() 或者 re.sub() 的简单替换操作能删除或者改变指定的字符序列。 你同样还可以使用2.9小节的 unicodedata.normalize() 函数将unicode文本标准化。


然后，有时候你可能还想在清理操作上更进一步。比如，你可能想消除整个区间上的字符或者去除变音符。 为了这样做，你可以使用经常会被忽视的 str.translate() 方法。 为了演示，假设你现在有下面这个凌乱的字符串：

In [101]:
s = 'pýtĥöñ\fis\tawesome\r\n'

In [102]:
s

'pýtĥöñ\x0cis\tawesome\r\n'

In [103]:
remap = {
    ord('\t'): ' ',
    ord('\f'): ' ',
    ord('\r'): None # deleted
}

In [104]:
a = s.translate(remap)
a

'pýtĥöñ is awesome\n'

## 13. 字符串对齐

In [105]:
# 对于基本的字符串对齐操作，可以使用字符串的ljust(),rjust()和center()方法
text = 'Hello World'

In [106]:
text.ljust(20)

'Hello World         '

In [107]:
text.rjust(20)

'         Hello World'

In [108]:
text.center(20)

'    Hello World     '

In [109]:
# 所有这些方法都能接受一个可选的填充字符
text.rjust(20,'=')

'=========Hello World'

In [110]:
text.center(20,'*')

'****Hello World*****'

In [111]:
# 函数format()同样可以用来很容易的对齐字符串。你要做的就是使用<,>或者^字符后面紧跟
# 一个指定的宽度。
format(text,'>20')

'         Hello World'

In [112]:
format(text,'<20')

'Hello World         '

In [113]:
format(text,'^20')

'    Hello World     '

In [114]:
# 如果你想指定一个非空格的填充字符，将它写到对齐字符的前面即可
format(text,'=>20s')

'=========Hello World'

In [115]:
format(text,'*^20s')

'****Hello World*****'

In [116]:
# 当格式化多个值的时候，这些格式代码也可以被用在format()方法中
'{:>10s} {:>10s}'.format('Hello','World') 

'     Hello      World'

In [117]:
# format()函数的一个好处是它不仅适用于字符串。他可以用来格式化任何值，使得它非常的通用。
# 比如你可以用来格式化数字
x = 1.2345
format(x,'>10')

'    1.2345'

In [118]:
format(x,'^10')

'  1.2345  '

## 14. 合并拼接字符串

如果你想要合并的字符串是在一个序列或者iterable中，那么最快的方式就是使用join（）方法

In [119]:
parts = ['Is','Chicago','Not','Chicago?']

In [120]:
' '.join(parts)

'Is Chicago Not Chicago?'

In [121]:
','.join(parts)

'Is,Chicago,Not,Chicago?'

In [122]:
''.join(parts)

'IsChicagoNotChicago?'

>> 初看起来，这种语法比较怪，但是join()被指定为字符串的一个方法。这样做的部分原因是你想去连接的对象可能来自各种不同的数据序列（比如列表、元组、字典、文件、集合或生成器），如果在所有这些对象上都定义一个join()方法明显是冗余了。因此你只需要指定你性药的分割字符串并调用它的join()方法去将文本片段组合起来

In [123]:
# 若果你仅仅只是合并少数几个字符串，使用加号通常已经足够了
a = 'Is Chicago'
b = 'Not chicago'
a + ' ' + b

'Is Chicago Not chicago'

In [124]:
# 加号操作符在作为一些复杂字符串何时话的替代方案的时候通常也工作的很好
print('{} {}'.format(a,b))

Is Chicago Not chicago


In [125]:
print(a + ' ' + b)

Is Chicago Not chicago


In [126]:
# 如果你想在源码上将两个字面字符串合并起来，你只需要简单的将他们放到一起，不需要用加号
a = 'Hello' 'World'
a

'HelloWorld'

最重要的需要引起注意的是，当我们使用加号(+)操作符去连接大量的字符串的时候是非常低效率的， 因为加号连接会引起内存复制以及垃圾回收操作。 特别的，你永远都不应像下面这样写字符串连接代码：

In [127]:
s = ''
for p in parts:
    s += p

这种写法会比使用 join() 方法运行的要慢一些，因为每一次执行+=操作的时候会创建一个新的字符串对象。 你最好是先收集所有的字符串片段然后再将它们连接起来。

In [128]:
data = ['ACME',50,91.1]

In [129]:
','.join(str(d) for d in data)

'ACME,50,91.1'

同样还得注意不必要的字符串连接操作。有时候程序员在没有必要做连接操作的时候仍然多此一举。比如在打印的时候：

In [130]:
# print(a + ':' + b + ':' + c) # Ugly
# print(':'.join([a, b, c])) # Still ugly
# print(a, b, c, sep=':') # Better

最后谈一下，如果你准备编写构建大量小字符串的输出代码， 你最好考虑下使用生成器函数，利用yield语句产生输出片段。比如：

In [131]:
def sample():
    yield 'Is'
    yield 'Chicago'
    yield 'Not'
    yield 'Chicago?'

In [132]:
text = ''.join(sample())

In [133]:
text

'IsChicagoNotChicago?'

## 15. 字符串中插入变量

Python并没有对字符串中简单替换变量值提供直接饿支持。但是通过使用字符串的format()方法可以解决这个问题

In [134]:
s = '{name} has {n} messages. '

In [135]:
s.format(name='Guido',n = 17)

'Guido has 17 messages. '

In [136]:
# 或者如果要被替换的变量能在变量域中找到，那么你可以结合使用format_map()和vars()
name = "Guido"
n = 37
s.format_map(vars())

'Guido has 37 messages. '

In [137]:
# vars()还有一个有意思的特性就是它也适用于对象实例。
class Info:
    def __init__(self,name,n):
        self.name = name
        self.n = n

In [138]:
a = Info('Guido',37)

In [139]:
s.format_map(vars(a))

'Guido has 37 messages. '

In [140]:
#format()和format_map()的一个缺陷就是他们并不能很好的处理变量缺失的情况
s.format(name='Guido')

KeyError: 'n'

In [146]:
# 一种避免这种错误的方法是另外定义一个含有__missing__()方法的字典对象，就像这样
class safesub(dict):
    """防止key找不到"""
    def __missing__(self,key):
        return '{' + key + '}'

In [147]:
# 现在你可以利用这个类包装输入后传递给format_map()
del n

In [148]:
s.format_map(safesub(vars()))

AttributeError: 'bytes' object has no attribute 'format_map'

In [149]:
# 如果你发现自己在代码中频繁的执行这些步骤，你可以将变量替换步骤用一个工具函数封装起来。就像下面这样：
import sys

def sub(text):
    return text.format_map(safesub(sys._getframe(1).f_locals))

In [150]:
name = "Guido"

In [151]:
n = 37

In [152]:
print(sub('Hello {name}'))

Hello Guido


In [153]:
print(sub('You hava {n} messages.'))

You hava 37 messages.


In [154]:
print(sub("YOur favorite color is {color}"))

YOur favorite color is {color}


## 16. 以指定列宽格式化字符串

解决办法：使用textwrap模块来格式化字符串的输出。

In [155]:
s = "Look into my eyes, look into my eyes, the eyes, the eyes, \
the eyes, not around the eyes, don't look around the eyes, \
look into my eyes, you're under."


In [156]:
import textwrap

In [157]:
print(textwrap.fill(s,70))

Look into my eyes, look into my eyes, the eyes, the eyes, the eyes,
not around the eyes, don't look around the eyes, look into my eyes,
you're under.


In [158]:
print(textwrap.fill(s,40))

Look into my eyes, look into my eyes,
the eyes, the eyes, the eyes, not around
the eyes, don't look around the eyes,
look into my eyes, you're under.


In [159]:
print(textwrap.fill(s,40,initial_indent= '   '))

   Look into my eyes, look into my eyes,
the eyes, the eyes, the eyes, not around
the eyes, don't look around the eyes,
look into my eyes, you're under.


In [160]:
print(textwrap.fill(s,40,subsequent_indent='   '))

Look into my eyes, look into my eyes,
   the eyes, the eyes, the eyes, not
   around the eyes, don't look around
   the eyes, look into my eyes, you're
   under.


>> textwrap 模块对于字符串打印是非常有用的，特别是当你希望输出自动匹配终端大小的时候。 你可以使用 os.get_terminal_size() 方法来获取终端的大小尺寸。比如：

## 17. 在字符串中处理html和xml

如果你想替换文本字符串中的 ‘<’ 或者 ‘>’ ，使用 html.escape() 函数可以很容易的完成。比如：

In [161]:
 s = 'Elements are written as "<tag>text</tag>".'

In [162]:
import html

In [163]:
print(s)

Elements are written as "<tag>text</tag>".


In [164]:
print(html.escape(s))

Elements are written as &quot;&lt;tag&gt;text&lt;/tag&gt;&quot;.


In [165]:
# disable escaping of quotes
print(html.escape(s,quote=False))

Elements are written as "&lt;tag&gt;text&lt;/tag&gt;".


如果你正在处理的是ASCII文本，并且想将非ASCII文本对应的编码实体嵌入进去， 可以给某些I/O函数传递参数 errors='xmlcharrefreplace' 来达到这个目。比如：

In [166]:
s = 'Spicy Jalapeño'

In [167]:
s.encode('ascii',errors='xmlcharrefreplace')

b'Spicy Jalape&#241;o'

>> 在生成HTML或者XML文本的时候，如果正确的转换特殊标记字符是一个很容易被忽视的细节。 特别是当你使用 print() 函数或者其他字符串格式化来产生输出的时候。 使用像 html.escape() 的工具函数可以很容易的解决这类问题。

>>如果你想以其他方式处理文本，还有一些其他的工具函数比如 xml.sax.saxutils.unescapge() 可以帮助你。 然而，你应该先调研清楚怎样使用一个合适的解析器。 比如，如果你在处理HTML或XML文本， 使用某个解析模块比如 html.parse 或 xml.etree.ElementTree 已经帮你自动处理了相关的替换细节。

## 18. 字符串令牌解析

In [168]:
text = 'foo = 23 + 42 * 10'

In [169]:
# 对于令牌化字符串，不仅需要匹配模式，还得指定模式的类型。比如你可能想将字符串想下面这样
# 转换为序列对
tokens = [('NAME', 'foo'), ('EQ','='), ('NUM', '23'), ('PLUS','+'),
          ('NUM', '42'), ('TIMES', '*'), ('NUM', '10')]

In [170]:
import re

In [171]:
NAME = r'(?P<NAME>[a-zA-Z_][a-zA-Z_0-9]*)'
NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
TIMES = r'(?P<TIMES>\*)'
EQ = r'(?P<EQ>=)'
WS = r'(?P<WS>\s+)'

master_pat = re.compile('|'.join([NAME,NUM,PLUS,TIMES,EQ,WS]))

>> 上面的模式中， `?P<TOKENINAME>`用于给一个模式命名，供后面使用。

下一步，为了令牌化，使用模式对象很少被人知道的 scanner() 方法。 这个方法会创建一个 scanner 对象， 在这个对象上不断的调用 match() 方法会一步步的扫描目标文本，每步一个匹配。 下面是演示一个 scanner 对象如何工作的交互式例子：

In [172]:
scanner = master_pat.scanner('foo = 42')

In [173]:
scanner.match()

<re.Match object; span=(0, 3), match='foo'>

In [174]:
_.lastgroup,_.group()

('NAME', 'foo')

In [175]:
scanner.match()

<re.Match object; span=(3, 4), match=' '>

In [176]:
_.lastgroup,_.group()

('WS', ' ')

In [177]:
# 实际使用中这种技术的时候，可以很容易的像下面这样将上述代码打包到一个生成器中
from collections import namedtuple

def generate_tokens(pat,text):
    Token = namedtuple('Token',['type','value'])
    scanner = pat.scanner(text)
    for m in iter(scanner.match,None):
        yield Token(m.lastgroup,m.group())
        
# example use
for tok in generate_tokens(master_pat,'foo = 42'):
    print(tok)

Token(type='NAME', value='foo')
Token(type='WS', value=' ')
Token(type='EQ', value='=')
Token(type='WS', value=' ')
Token(type='NUM', value='42')


>> 通常来讲令牌化是很多高级文本解析与处理的第一步。 为了使用上面的扫描方法，你需要记住这里一些重要的几点。 第一点就是你必须确认你使用正则表达式指定了所有输入中可能出现的文本序列。 如果有任何不可匹配的文本出现了，扫描就会直接停止。这也是为什么上面例子中必须指定空白字符令牌的原因。

## 19. 实现一个简单的递归下降分析器

在这个问题中，我们集中讨论根据特殊语法去解析文本的问题。 为了这样做，你首先要以BNF或者EBNF形式指定一个标准语法。 比如，一个简单数学表达式语法可能像下面这样：

```
expr ::= expr + term
    |   expr - term
    |   term

term ::= term * factor
    |   term / factor
    |   factor

factor ::= ( expr )
    |   NUM

```
或者使用

```

expr ::= term { (+|-) term }*

term ::= factor { (*|/) factor }*

factor ::= ( expr )
    |   NUM


````

In [178]:
import re
import collections

# Token specification
NUM = r'(?P<NUM>\d+)'
PLUS = r'(?P<PLUS>\+)'
MINUS = r'(?P<MINUS>-)'
TIMES = r'(?P<TIMES>\*)'
DIVIDE = r'(?P<DIVIDE>/)'
LPAREN = r'(?P<LPAREN>\()'
RPAREN = r'(?P<RPAREN>\))'
WS = r'(?P<WS>\s+)'

master_pat = re.compile('|'.join([NUM, PLUS, MINUS, TIMES,
                                  DIVIDE, LPAREN, RPAREN, WS]))
# Tokenizer
Token = collections.namedtuple('Token', ['type', 'value'])


def generate_tokens(text):
    scanner = master_pat.scanner(text)
    for m in iter(scanner.match, None):
        tok = Token(m.lastgroup, m.group())
        if tok.type != 'WS':
            yield tok

# Parser
class ExpressionEvaluator:
    '''
    Implementation of a recursive descent parser. Each method
    implements a single grammar rule. Use the ._accept() method
    to test and accept the current lookahead token. Use the ._expect()
    method to exactly match and discard the next token on on the input
    (or raise a SyntaxError if it doesn't match).
    '''

    def parse(self, text):
        self.tokens = generate_tokens(text)
        self.tok = None  # Last symbol consumed
        self.nexttok = None  # Next symbol tokenized
        self._advance()  # Load first lookahead token
        return self.expr()

    def _advance(self):
        'Advance one token ahead'
        self.tok, self.nexttok = self.nexttok, next(self.tokens, None)

    def _accept(self, toktype):
        'Test and consume the next token if it matches toktype'
        if self.nexttok and self.nexttok.type == toktype:
            self._advance()
            return True
        else:
            return False

    def _expect(self, toktype):
        'Consume next token if it matches toktype or raise SyntaxError'
        if not self._accept(toktype):
            raise SyntaxError('Expected ' + toktype)

    # Grammar rules follow
    def expr(self):
        "expression ::= term { ('+'|'-') term }*"
        exprval = self.term()
        while self._accept('PLUS') or self._accept('MINUS'):
            op = self.tok.type
            right = self.term()
            if op == 'PLUS':
                exprval += right
            elif op == 'MINUS':
                exprval -= right
        return exprval

    def term(self):
        "term ::= factor { ('*'|'/') factor }*"
        termval = self.factor()
        while self._accept('TIMES') or self._accept('DIVIDE'):
            op = self.tok.type
            right = self.factor()
            if op == 'TIMES':
                termval *= right
            elif op == 'DIVIDE':
                termval /= right
        return termval

    def factor(self):
        "factor ::= NUM | ( expr )"
        if self._accept('NUM'):
            return int(self.tok.value)
        elif self._accept('LPAREN'):
            exprval = self.expr()
            self._expect('RPAREN')
            return exprval
        else:
            raise SyntaxError('Expected NUMBER or LPAREN')


def descent_parser():
    e = ExpressionEvaluator()
    print(e.parse('2'))
    print(e.parse('2 + 3'))
    print(e.parse('2 + 3 * 4'))
    print(e.parse('2 + (3 + 4) * 5'))
    # print(e.parse('2 + (3 + * 4)'))
    # Traceback (most recent call last):
    #    File "<stdin>", line 1, in <module>
    #    File "exprparse.py", line 40, in parse
    #    return self.expr()
    #    File "exprparse.py", line 67, in expr
    #    right = self.term()
    #    File "exprparse.py", line 77, in term
    #    termval = self.factor()
    #    File "exprparse.py", line 93, in factor
    #    exprval = self.expr()
    #    File "exprparse.py", line 67, in expr
    #    right = self.term()
    #    File "exprparse.py", line 77, in term
    #    termval = self.factor()
    #    File "exprparse.py", line 97, in factor
    #    raise SyntaxError("Expected NUMBER or LPAREN")
    #    SyntaxError: Expected NUMBER or LPAREN


if __name__ == '__main__':
    descent_parser()

2
5
14
37


## 20. 字节字符串上的字符串操作

In [179]:
data = b'Hello World'

In [180]:
data[0:5]

b'Hello'

In [181]:
data.startswith(b'Hello')

True

In [182]:
data.split()

[b'Hello', b'World']

In [183]:
data.replace(b'Hello',b'Hello Cruel')

b'Hello Cruel World'

In [184]:
# 这些操作同样也适用于字节数组，比如：
data = bytearray(b'Hello World')

In [185]:
data[0:5]

bytearray(b'Hello')

In [186]:
data.startswith(b'Hello')

True

In [187]:
data.split()

[bytearray(b'Hello'), bytearray(b'World')]

In [188]:
data.replace(b'Hello',b'Hello Cruel')

bytearray(b'Hello Cruel World')

大多数情况下，在文本字符串上的操作均可用于字节字符串。 然而，这里也有一些需要注意的不同点。首先，字节字符串的索引操作返回整数而不是单独字符。比如：

In [189]:
# Text string
a = 'Hello World'
a[0]

'H'

In [190]:
a[1]

'e'

In [191]:
# Byte string
b = b'Hello World'
b[0]

72

In [192]:
b[1]

101

第二点，字节字符串不会提供一个美观的字符串表示，也不能很好的打印出来，除非它们先被解码为一个文本字符串。比如：

In [193]:
s = b'Hello World'

In [194]:
print(s)

b'Hello World'


In [195]:
print(s.decode('ascii'))

Hello World


>> 最后提一点，一些程序员为了提升程序执行的速度会倾向于使用字节字符串而不是文本字符串。 尽管操作字节字符串确实会比文本更加高效(因为处理文本固有的Unicode相关开销)。 这样做通常会导致非常杂乱的代码。你会经常发现字节字符串并不能和Python的其他部分工作的很好， 并且你还得手动处理所有的编码/解码操作。 坦白讲，如果你在处理文本的话，就直接在程序中使用普通的文本字符串而不是字节字符串。不做死就不会死！